In [1]:
# import Libraries

import cohere
import langchain
import pinecone 
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
#from langchain.llms import Llama3

In [1]:
%pip install \
    "pinecone[grpc]" \
    "langchain-pinecone" \
    "langchain-openai" \
    "langchain-text-splitters" \
    "langchain"

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [12]:
# ## Lets Read the document
# def read_doc(directory):
#     file_loader=PyPDFDirectoryLoader(directory)
#     documents=file_loader.load(directory)
#     return documents

In [5]:
from langchain.document_loaders import PyPDFLoader

def read_doc(file_path):
    file_loader = PyPDFLoader(file_path)
    document = file_loader.load()
    return document

# Load the single PDF
doc = read_doc("Black Holes Explained.pdf")

# Check the length of the document
len(doc)


92

In [6]:
## Divide the docs into chunks
### https://api.python.langchain.com/en/latest/text_splitter/langchain.text_splitter.RecursiveCharacterTextSplitter.html#
def chunk_data(docs,chunk_size=800,chunk_overlap=50):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=chunk_overlap)
    doc=text_splitter.split_documents(docs)
    return doc

In [14]:
!pip install langchain-huggingface

from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings




ModuleNotFoundError: No module named 'langchain_huggingface'

^C


ModuleNotFoundError: No module named 'langchain_huggingface'

In [9]:
api_key="a5ed3c1e-9541-431b-9b10-5d6ae6876eb9"

In [10]:
from pinecone.grpc import PineconeGRPC as Pinecone
import os
from pinecone import Pinecone,ServerlessSpec
index_name="rag-getting-started"
## initialize the Pinecone client
pc=Pinecone(api_key=api_key)

#create the index
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1024,  # dimensionality of dense model
        metric="cosine",  # sparse values supported only for dotproduct
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

In [11]:
index=pc.Index(index_name)
index

In [12]:
from langchain.vectorstores import Pinecone as PC
#index=Pinecone.from_documents(doc,embeddings,index_name=index_name)
index = PC.from_documents(doc, embeddings, index_name=index_name)

NameError: name 'embeddings' is not defined

In [56]:
## Cosine Similarity Retreive Results from VectorDB
def retrieve_query(query,k=2):
    matching_results=index.similarity_search(query,k=k)
    return matching_results

In [58]:
# from transformers import AutoModelForQuestionAnswering, AutoTokenizer
# from langchain.chains.question_answering import load_qa_chain

# model_name = "bert-base-uncased-squad2"
# model = AutoModelForQuestionAnswering.from_pretrained(model_name)
# tokenizer = AutoTokenizer.from_pretrained(model_name)

# qa_chain = load_qa_chain(model, tokenizer)

In [62]:
# from bloom_model import load_bloom
# from langchain.chains.question_answering import load_qa_chain

# model = load_bloom("bloom-560m")
# qa_chain = load_qa_chain(model)
from llama_index import LlamaIndex
from langchain.chains.question_answering import load_qa_chain

index = LlamaIndex.from_documents(...)
qa_chain = load_qa_chain(index)


ImportError: cannot import name 'LlamaIndex' from 'llama_index' (unknown location)